In [1]:
from yahoofinancials import YahooFinancials

In [2]:
from datetime import date, timedelta
from datetime import datetime

In [21]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
SP500 = ['AAPL', 'MSFT', 'BRK-B']

In [5]:
yf_SP500 = YahooFinancials(SP500)

In [6]:
prices = yf_SP500.get_historical_price_data('2020-03-05', "2020-04-06", 
                                            "daily")

In [7]:
prices['AAPL']['prices'][0]

{'date': 1583418600,
 'high': 74.88749694824219,
 'low': 72.85250091552734,
 'open': 73.87999725341797,
 'close': 73.2300033569336,
 'volume': 187572800,
 'adjclose': 72.6667251586914,
 'formatted_date': '2020-03-05'}

In [8]:
[[p['formatted_date'], p['close']] for p in prices['AAPL']['prices']]

[['2020-03-05', 73.2300033569336],
 ['2020-03-06', 72.25749969482422],
 ['2020-03-09', 66.5425033569336],
 ['2020-03-10', 71.33499908447266],
 ['2020-03-11', 68.85749816894531],
 ['2020-03-12', 62.057498931884766],
 ['2020-03-13', 69.49250030517578],
 ['2020-03-16', 60.5525016784668],
 ['2020-03-17', 63.21500015258789],
 ['2020-03-18', 61.66749954223633],
 ['2020-03-19', 61.19499969482422],
 ['2020-03-20', 57.310001373291016],
 ['2020-03-23', 56.092498779296875],
 ['2020-03-24', 61.720001220703125],
 ['2020-03-25', 61.380001068115234],
 ['2020-03-26', 64.61000061035156],
 ['2020-03-27', 61.935001373291016],
 ['2020-03-30', 63.70249938964844],
 ['2020-03-31', 63.5724983215332],
 ['2020-04-01', 60.227500915527344],
 ['2020-04-02', 61.23249816894531],
 ['2020-04-03', 60.352500915527344]]

In [9]:
def get_prices(event_date, stocks, intervals=[1, 7, 28]):
    """Output next closing price, 
    closing price after a week, 
    and closing price after a month
    """
    future_dates = [
        str(datetime.strptime(event_date,'%Y-%m-%d').date() + timedelta(i+1)) for i in intervals
    ]
    
    output_dict = {stock: None for stock in stocks }
    yf = YahooFinancials(stocks)
    
    price_all = yf.get_historical_price_data(event_date, future_dates[-1], "daily")
    for stock in stocks:
        prices = [price_all[stock]['prices'][0]['open']]
        prices += [price_all[stock]['prices'][i-1]['close'] for i in intervals[:-1]]
        prices += [price_all[stock]['prices'][-1]['close']]
        output_dict[stock] = prices
    
    return output_dict

In [10]:
get_prices('2020-03-05', ['AAPL', 'MSFT', 'MMM', 'ABT', 'ABBV'])

{'AAPL': [73.87999725341797,
  73.2300033569336,
  69.49250030517578,
  61.23249816894531],
 'MSFT': [166.0500030517578,
  166.27000427246094,
  158.8300018310547,
  155.25999450683594],
 'MMM': [150.8000030517578,
  151.41000366210938,
  141.67999267578125,
  137.91000366210938],
 'ABT': [80.29000091552734,
  80.5199966430664,
  81.6500015258789,
  79.44000244140625],
 'ABBV': [90.05999755859375,
  90.61000061035156,
  85.37000274658203,
  75.12999725341797]}

In [11]:
stocks = list(pd.read_csv('../output/SP500_base.csv').Symbol)

In [12]:
stocks[0]

'MMM'

In [13]:
stocks[65] = 'BRK-B'
stocks[78] = 'BF-B'

In [15]:
prices = get_prices('2021-04-09', stocks, 
                    intervals=[1, 7])

In [16]:
len(prices)

505

In [17]:
prices_pd = pd.DataFrame(prices)

In [18]:
#prices_pd.to_csv('../output/SP500_20210409.csv', index=False)

In [27]:
prices_pd = pd.read_csv('../output/SP500_20210409.csv')

In [28]:
# (open next day
#  close next day
#  close next week)
#  2021/04/10
prices_pd = prices_pd.transpose()

In [26]:
prices_pd.iloc[:, 1]

MMM     198.000000
ABT     120.900002
ABBV    107.540001
ABMD    333.809998
ACN     287.709991
           ...    
YUM     114.959999
ZBRA    507.510010
ZBH     169.479996
ZION     55.939999
ZTS     159.850006
Name: 1, Length: 505, dtype: float64

In [30]:
prices_pd['change_day'] = (prices_pd.iloc[:, 1] - prices_pd.iloc[:, 0]) / prices_pd.iloc[:, 0] * 100
prices_pd['change_week'] = (prices_pd.iloc[:, 2] - prices_pd.iloc[:, 0]) / prices_pd.iloc[:, 0] * 100

In [305]:
prices_pd
#.to_csv('future_prices_20210410.csv', index=True)

In [80]:
SP500_sentiments = pd.read_csv('../output/sentiment_full_20210409.csv')

In [81]:
SP500_sentiments.head()

,created_at,Security,text,sentiment
0,Fri Apr 09 19:48:19 +0000 2021,3M Company,3M sues Florida company over sale of fake N95 ...,-0.4767
1,Fri Apr 09 19:48:08 +0000 2021,3M Company,3M sues Florida company over sale of fake N95 ...,-0.4767
2,Fri Apr 09 19:46:58 +0000 2021,3M Company,Great installation photos of 3M Fireblock Foam...,0.8519
3,Fri Apr 09 19:42:59 +0000 2021,3M Company,RT @news4buffalo: Product manufacturer 3M has ...,-0.7845
4,Fri Apr 09 19:39:02 +0000 2021,3M Company,Product manufacturer 3M has filed a trademark ...,-0.7845


In [192]:
SP500_sentiments.shape

(549, 4)

## Update the sentiment vector

In [291]:
stock_list = pd.read_csv('../output/SP500_base.csv')[['Symbol', 'Security']]

In [292]:
SP500_matrix = np.load('../output/matrix2.npy')

In [293]:
stock_list['sentiment'] = 0

In [294]:
def update_sentiment(event, stocks, distance_matrix, weight=0.01):
    """Inplace edit `stocks.sentiment` to save memory"""
    # assume single stock
    stock = stocks[stocks.Security==event.Security].Symbol
    index = stock.index[0]
    
    # update first order nodes
    # assume `sentiment` is last column
    stocks.iat[idx, -1] += event.sentiment
    
    # update second order nodes
    stocks.sentiment += stocks.sentiment @ distance_matrix
    
    # reweight
    if np.sum(stocks.sentiment != 0):
        stocks.sentiment *= (500/np.sum(stocks.sentiment))

In [295]:
stock_list.head()

,Symbol,Security,sentiment
0,MMM,3M Company,0
1,ABT,Abbott Laboratories,0
2,ABBV,AbbVie Inc.,0
3,ABMD,Abiomed,0
4,ACN,Accenture,0


In [296]:
for _, row in SP500_sentiments.iterrows():
    update_sentiment(row, stock_list, SP500_matrix)

In [297]:
stock_list

,Symbol,Security,sentiment
0,MMM,3M Company,1.332609
1,ABT,Abbott Laboratories,0.436241
2,ABBV,AbbVie Inc.,0.436241
3,ABMD,Abiomed,0.471615
4,ACN,Accenture,3.217598
...,...,...,...
500,YUM,Yum! Brands Inc,0.614465
501,ZBRA,Zebra Technologies,2.813353
502,ZBH,Zimmer Biomet,0.436241
503,ZION,Zions Bancorp,0.861769


In [282]:
# stock_list.to_csv('../output/sentiment_vector_20210410.csv', index=False)

## Correlation

In [ ]:
# For 2021/04/10

In [304]:
np.corrcoef([stock_list.sentiment, 
             prices_pd.change_day, 
             prices_pd.change_week])

array([[ 1.        ,  0.14731545, -0.032121  ],
       [ 0.14731545,  1.        ,  0.35507166],
       [-0.032121  ,  0.35507166,  1.        ]])

## One event example

In [314]:
event = SP500_sentiments.iloc[2, :]

In [315]:
event.text

'Great installation photos of 3M Fireblock Foam and CP 25 supplied by General Insulation Company.  Special thank you… https://t.co/NiGsi7TsZX'

In [316]:
event.sentiment

0.8519

In [322]:
sentiment_original_array = np.zeros(505)
sentiment_original_array[0] = event.sentiment

In [320]:
sentiment_original_array[0] = event.sentiment

In [325]:
stock_list = pd.read_csv('../output/SP500_base.csv')[['Symbol']]
stock_list['sentiment'] = 0

In [327]:
impact = sentiment_original_array @ SP500_matrix

In [328]:
stock_list.sentiment = impact

In [332]:
stock_list.sentiment *= 500/np.sum(stock_list.sentiment)

In [334]:
stock_list

,Symbol,sentiment
0,MMM,16.853887
1,ABT,0.000005
2,ABBV,0.000005
3,ABMD,0.000026
4,ACN,0.000014
...,...,...
500,YUM,-0.000044
501,ZBRA,0.000006
502,ZBH,0.000005
503,ZION,-0.000035


In [335]:
stock_list.to_csv('../output/event_example.csv', index=False)